# 📊 Statistiques de Recrutement par Client

Ce notebook génère des statistiques détaillées sur les candidatures pour un client spécifique, en analysant toutes les campagnes et candidatures liées à ce client.

## 🎯 Objectif

Ce notebook permet de :
- Identifier automatiquement un client par son nom (groupe, marque ou unité)
- Récupérer toutes les campagnes liées à ce client
- Analyser toutes les candidatures pour ces campagnes
- Générer des statistiques détaillées (par marque, unité, campagne, source, statut)
- Exporter les résultats dans un fichier Excel structuré

## 📋 Guide d'utilisation

### Étape 1 : Configuration

1. **Modifiez la variable `COMPANY_NAME`** dans la cellule de configuration (Étape 2)
   - Exemple : `COMPANY_NAME = "critInterim"`
   - Le matching est insensible à la casse et tolère les correspondances partielles

### Étape 2 : Exécution

**Méthode rapide :**
- Menu **"Runtime"** → **"Run all"** (Exécuter tout)

**Méthode manuelle :**
- Exécutez chaque cellule dans l'ordre (Shift + Entrée)

### Étape 3 : Résultats

Un fichier Excel sera généré avec plusieurs feuilles contenant toutes les statistiques.

---

## ⚠️ Attention

**Temps d'exécution :** Ce notebook peut prendre plusieurs minutes si le client a beaucoup de campagnes et candidatures. Ne fermez pas l'onglet pendant l'exécution.

**Données :** Toutes les données sont récupérées depuis l'API en temps réel. Les résultats reflètent l'état actuel des données.

---

**Prêt à commencer ? Configurez le nom du client ci-dessous ! 👇**


## 📦 Étape 1 : Installation des bibliothèques nécessaires

Cette cellule installe tous les outils nécessaires pour faire fonctionner le notebook.

**⚠️ Important :** Exécutez cette cellule en premier et attendez qu'elle se termine avant de continuer !

**Temps estimé :** 10-30 secondes


In [67]:
%pip install pandas openpyxl requests numpy


Note: you may need to restart the kernel to use updated packages.


## 📚 Étape 2 : Importation des bibliothèques

Cette cellule charge les outils Python nécessaires pour le traitement des données.

**Aucune action requise** - Exécutez simplement la cellule.


In [68]:
import requests
import pandas as pd
import numpy as np
import json
import re
import warnings
import os
from datetime import datetime
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

warnings.filterwarnings('ignore')


## ⚙️ Étape 3 : Configuration

**⚠️ IMPORTANT :** Modifiez la variable `COMPANY_NAME` ci-dessous avec le nom du client que vous souhaitez analyser.

**Exemples de noms de clients :**
- `"critInterim"` - Pour analyser Crit Interim
- `"E.Leclerc"` - Pour analyser E.Leclerc
- `"Decathlon"` - Pour analyser Decathlon

**Note :** Le matching est insensible à la casse et tolère les correspondances partielles. Par exemple, "crit" trouvera "Crit Interim", "CRIT INTERIM", etc.


In [69]:
# ============================================
# CONFIGURATION - MODIFIEZ ICI
# ============================================

# Nom du client à analyser (modifiez cette variable)
COMPANY_NAME = "CRIT INTERIM"  # ← MODIFIEZ ICI

# Configuration de l'API
API_URL = "https://api.smart-process-rh.com/v1/"
API_KEY = "9TTaz70w8biMjvJ9Q5eIHZwVlQRNmjqAqiNzyGjfeI1S4nubpkSAL1h87FoNrlMv"  # Remplacez par votre clé API si nécessaire

# Headers pour les requêtes API
HEADERS = {
    "x-api-key": API_KEY
}

# Endpoints de l'API
ENDPOINTS = {
    "groups": f"{API_URL}/debug/groups",
    "brands": f"{API_URL}/debug/brands",
    "units": f"{API_URL}/debug/units",
    "campaigns": f"{API_URL}/debug/campaigns",
    "applications": f"{API_URL}/debug/applications"
}

print("✅ Configuration chargée :")
print(f"   Nom du client recherché: '{COMPANY_NAME}'")
print(f"   URL de l'API: {API_URL}")
print(f"\n💡 Astuce : Pour analyser un autre client, modifiez la variable COMPANY_NAME ci-dessus.")


✅ Configuration chargée :
   Nom du client recherché: 'CRIT INTERIM'
   URL de l'API: https://api.smart-process-rh.com/v1/

💡 Astuce : Pour analyser un autre client, modifiez la variable COMPANY_NAME ci-dessus.


## 📥 Étape 4 : Récupération des données de référence

Cette cellule récupère toutes les données de référence nécessaires :
- **Groupes** : Les groupes d'entreprises
- **Marques** : Les marques sous chaque groupe
- **Unités** : Les unités sous chaque marque

Ces données permettent d'identifier la hiérarchie complète du client recherché.

**Temps estimé :** 10-20 secondes


In [70]:
def fetch_data(endpoint, headers):
    """Récupère les données depuis un endpoint API"""
    try:
        response = requests.get(endpoint, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Erreur {response.status_code} pour {endpoint}")
            return []
    except Exception as e:
        print(f"❌ Erreur lors de la récupération de {endpoint}: {e}")
        return []

def extract_data_column(df):
    """Extrait les données de la colonne 'data' si elle existe"""
    if df.empty:
        return df
    
    if 'data' in df.columns:
        # Extraire les dictionnaires de la colonne 'data'
        data_list = []
        for idx, row in df.iterrows():
            if pd.notna(row['data']) and isinstance(row['data'], dict):
                data_list.append(row['data'])
        return pd.DataFrame(data_list) if data_list else pd.DataFrame()
    else:
        return df

# Récupération des groupes
print("📥 Récupération des groupes...")
groups_data = fetch_data(ENDPOINTS["groups"], HEADERS)
groups_df = pd.DataFrame(groups_data) if groups_data else pd.DataFrame()
groups_df = extract_data_column(groups_df)
print(f"✅ {len(groups_df)} groupes récupérés")

# Récupération des marques
print("\n📥 Récupération des marques...")
brands_data = fetch_data(ENDPOINTS["brands"], HEADERS)
brands_df = pd.DataFrame(brands_data) if brands_data else pd.DataFrame()
brands_df = extract_data_column(brands_df)
print(f"✅ {len(brands_df)} marques récupérées")

# Récupération des unités
print("\n📥 Récupération des unités...")
units_data = fetch_data(ENDPOINTS["units"], HEADERS)
units_df = pd.DataFrame(units_data) if units_data else pd.DataFrame()
units_df = extract_data_column(units_df)
print(f"✅ {len(units_df)} unités récupérées")

# Aperçu des données
if len(groups_df) > 0:
    print(f"\n📊 Aperçu des groupes (premiers résultats):")
    if 'name' in groups_df.columns:
        print(groups_df[['id', 'name']].head() if 'id' in groups_df.columns else groups_df[['name']].head())
    else:
        print(groups_df.head())
    print(f"\n💡 Structure des données - Colonnes disponibles: {list(groups_df.columns)}")


📥 Récupération des groupes...
✅ 23 groupes récupérés

📥 Récupération des marques...
✅ 51 marques récupérées

📥 Récupération des unités...
✅ 31 unités récupérées

📊 Aperçu des groupes (premiers résultats):
   id          name
0   2  Smart Profil
1   8    OptiMarché
2  10   Optimarché 
3  11       Eurocrm
4  13        Cibli 

💡 Structure des données - Colonnes disponibles: ['id', 'name', 'description', 'client_id', 'created_at', 'updated_at']


## 🔍 Étape 5 : Résolution de la portée du client

Cette cellule identifie automatiquement tous les groupes, marques et unités qui correspondent au nom du client recherché.

**Logique de matching :**
- Si le nom correspond à un **groupe**, toutes les marques et unités sous ce groupe sont incluses
- Si le nom correspond à une **marque**, le groupe parent et toutes les unités de cette marque sont incluses
- Si le nom correspond à une **unité**, le groupe et la marque parents sont incluses
- Le matching est **insensible à la casse** et tolère les **correspondances partielles**

**Résultat :** Une liste complète de tous les IDs (groupes, marques, unités) à analyser.


In [71]:
def normalize_name(name):
    """Normalise un nom pour la comparaison (minuscules, suppression de tous les espaces et caractères spéciaux)"""
    if pd.isna(name) or name is None:
        return ""
    # Convertir en minuscules, supprimer tous les espaces, tirets, points, etc.
    normalized = str(name).lower().strip()
    # Supprimer tous les caractères non-alphanumériques (espaces, tirets, points, etc.)
    normalized = re.sub(r'[^a-z0-9]', '', normalized)
    return normalized

def name_matches(search_name, target_name):
    """Vérifie si le nom recherché correspond au nom cible (insensible à la casse, correspondance partielle)"""
    search_normalized = normalize_name(search_name)
    target_normalized = normalize_name(target_name)
    
    # Si l'un est vide, pas de correspondance
    if not search_normalized or not target_normalized:
        return False
    
    # Correspondance bidirectionnelle : recherche dans cible OU cible dans recherche
    # Cela permet de trouver "crit" dans "critinterim" et "critinterim" dans "crit interim - rezé"
    return search_normalized in target_normalized or target_normalized in search_normalized

# Initialiser les listes d'IDs à inclure
matched_group_ids = []
matched_brand_ids = []
matched_unit_ids = []

# Dictionnaires pour stocker les relations parent-enfant
group_to_brands = {}
brand_to_units = {}
unit_to_brand = {}
unit_to_group = {}
brand_to_group = {}

# Construire les relations hiérarchiques
if len(brands_df) > 0 and 'group' in brands_df.columns:
    for _, brand in brands_df.iterrows():
        brand_id = brand.get('id')
        group_id = brand.get('group', {}).get('id') if isinstance(brand.get('group'), dict) else None
        if brand_id and group_id:
            if group_id not in group_to_brands:
                group_to_brands[group_id] = []
            group_to_brands[group_id].append(brand_id)
            brand_to_group[brand_id] = group_id

if len(units_df) > 0:
    for _, unit in units_df.iterrows():
        unit_id = unit.get('id')
        brand_id = unit.get('brand', {}).get('id') if isinstance(unit.get('brand'), dict) else None
        group_id = unit.get('group', {}).get('id') if isinstance(unit.get('group'), dict) else None
        
        if unit_id and brand_id:
            if brand_id not in brand_to_units:
                brand_to_units[brand_id] = []
            brand_to_units[brand_id].append(unit_id)
            unit_to_brand[unit_id] = brand_id
        
        if unit_id and group_id:
            unit_to_group[unit_id] = group_id

# Rechercher les correspondances dans les groupes
matched_groups = []
if len(groups_df) > 0 and 'name' in groups_df.columns:
    for _, group in groups_df.iterrows():
        group_name = group.get('name')
        if group_name and name_matches(COMPANY_NAME, group_name):
            group_id = group.get('id')
            matched_groups.append({'id': group_id, 'name': group_name, 'type': 'group'})
            matched_group_ids.append(group_id)
            # Inclure toutes les marques de ce groupe
            if group_id in group_to_brands:
                matched_brand_ids.extend(group_to_brands[group_id])

# Rechercher les correspondances dans les marques
matched_brands = []
if len(brands_df) > 0 and 'name' in brands_df.columns:
    for _, brand in brands_df.iterrows():
        brand_name = brand.get('name')
        if brand_name and name_matches(COMPANY_NAME, brand_name):
            brand_id = brand.get('id')
            matched_brands.append({'id': brand_id, 'name': brand_name, 'type': 'brand'})
            if brand_id not in matched_brand_ids:
                matched_brand_ids.append(brand_id)
            # Inclure le groupe parent
            if brand_id in brand_to_group:
                parent_group_id = brand_to_group[brand_id]
                if parent_group_id not in matched_group_ids:
                    matched_group_ids.append(parent_group_id)
            # Inclure toutes les unités de cette marque
            if brand_id in brand_to_units:
                matched_unit_ids.extend(brand_to_units[brand_id])

# Rechercher les correspondances dans les unités
matched_units = []
if len(units_df) > 0 and 'name' in units_df.columns:
    for _, unit in units_df.iterrows():
        unit_name = unit.get('name')
        if unit_name and name_matches(COMPANY_NAME, unit_name):
            unit_id = unit.get('id')
            matched_units.append({'id': unit_id, 'name': unit_name, 'type': 'unit'})
            if unit_id not in matched_unit_ids:
                matched_unit_ids.append(unit_id)
            # Inclure la marque parente
            if unit_id in unit_to_brand:
                parent_brand_id = unit_to_brand[unit_id]
                if parent_brand_id not in matched_brand_ids:
                    matched_brand_ids.append(parent_brand_id)
            # Inclure le groupe parent
            if unit_id in unit_to_group:
                parent_group_id = unit_to_group[unit_id]
                if parent_group_id not in matched_group_ids:
                    matched_group_ids.append(parent_group_id)

# Créer un résumé
summary_data = []
for match in matched_groups + matched_brands + matched_units:
    summary_data.append(match)

summary_df = pd.DataFrame(summary_data) if summary_data else pd.DataFrame()

print("=" * 60)
print("📊 RÉSUMÉ DE LA RÉSOLUTION DU CLIENT")
print("=" * 60)
print(f"\n🔍 Nom recherché: '{COMPANY_NAME}'")
print(f"\n✅ Correspondances trouvées:")
print(f"   - Groupes: {len(matched_groups)}")
print(f"   - Marques: {len(matched_brands)}")
print(f"   - Unités: {len(matched_units)}")

if len(summary_df) > 0:
    print(f"\n📋 Détail des correspondances:")
    print(summary_df.to_string(index=False))
else:
    print(f"\n⚠️ Aucune correspondance trouvée pour '{COMPANY_NAME}'")
    print(f"   Vérifiez l'orthographe ou essayez une correspondance partielle")

print(f"\n📊 Portée complète à analyser:")
print(f"   - Groupes IDs: {len(matched_group_ids)} ({matched_group_ids[:5]}{'...' if len(matched_group_ids) > 5 else ''})")
print(f"   - Marques IDs: {len(matched_brand_ids)} ({matched_brand_ids[:5]}{'...' if len(matched_brand_ids) > 5 else ''})")
print(f"   - Unités IDs: {len(matched_unit_ids)} ({matched_unit_ids[:5]}{'...' if len(matched_unit_ids) > 5 else ''})")
print("=" * 60)


📊 RÉSUMÉ DE LA RÉSOLUTION DU CLIENT

🔍 Nom recherché: 'CRIT INTERIM'

✅ Correspondances trouvées:
   - Groupes: 1
   - Marques: 2
   - Unités: 4

📋 Détail des correspondances:
 id                                                     name  type
 30                                             CRIT INTERIM group
149                                CRIT INTERIM Aéronautique brand
150                                CRIT INTERIM Région Ouest brand
 73                            CRIT INTERIM - Saint-Herblain  unit
 74                                      CRIT INTERIM - Rezé  unit
 75             CRIT INTERIM - Direction Régionale CRIT BPDL  unit
 76 CRIT INTERIM - Cabinet Experts & Cadres (Saint-Herblain)  unit

📊 Portée complète à analyser:
   - Groupes IDs: 1 ([30])
   - Marques IDs: 2 ([149, 150])
   - Unités IDs: 4 ([73, 74, 75, 76])


## 📋 Étape 6 : Récupération et filtrage des campagnes

Cette cellule :
- Récupère toutes les campagnes depuis l'API
- Filtre les campagnes appartenant aux groupes, marques ou unités identifiés
- Attache les noms lisibles (groupe, marque, unité) à chaque campagne

**Note :** Les noms de campagne sont reconstruits à partir de la hiérarchie (Groupe > Marque > Unité) pour faciliter la lecture.

**Temps estimé :** 15-30 secondes


In [72]:
# Récupération de toutes les campagnes
print("📥 Récupération de toutes les campagnes...")
campaigns_raw = fetch_data(ENDPOINTS["campaigns"], HEADERS)

# Créer un DataFrame comme pour les groupes
campaigns_df_raw = pd.DataFrame(campaigns_raw) if campaigns_raw else pd.DataFrame()

# Extraire les données de la colonne 'data' comme pour les groupes
campaigns_df = extract_data_column(campaigns_df_raw)

# Convertir en liste de dictionnaires pour le traitement
campaigns_data = campaigns_df.to_dict('records') if len(campaigns_df) > 0 else []

print(f"✅ {len(campaigns_data)} campagnes récupérées au total")

# Fonction pour vérifier si une campagne appartient à la portée du client
def campaign_belongs_to_scope(campaign, debug_campaign_id=None):
    """Vérifie si une campagne appartient à la portée du client
    Les endpoints debug utilisent directement group_id, brand_id, unit_id"""
    if not isinstance(campaign, dict):
        return False
    
    campaign_id = campaign.get('id')
    is_debug = debug_campaign_id is not None and campaign_id == debug_campaign_id
    
    # Vérifier le groupe (directement depuis group_id)
    group_id = campaign.get('group_id')
    if group_id and group_id in matched_group_ids:
        if is_debug:
            print(f"DEBUG Campaign {campaign_id}: Matched by group_id {group_id}")
        return True
    
    # Vérifier la marque (directement depuis brand_id)
    brand_id = campaign.get('brand_id')
    if brand_id and brand_id in matched_brand_ids:
        if is_debug:
            print(f"DEBUG Campaign {campaign_id}: Matched by brand_id {brand_id}")
        return True
    
    # Vérifier l'unité (directement depuis unit_id)
    unit_id = campaign.get('unit_id')
    if unit_id and unit_id in matched_unit_ids:
        if is_debug:
            print(f"DEBUG Campaign {campaign_id}: Matched by unit_id {unit_id}")
        return True
    
    if is_debug:
        print(f"DEBUG Campaign {campaign_id}: No match found")
        print(f"  - group_id: {group_id}")
        print(f"  - brand_id: {brand_id}")
        print(f"  - unit_id: {unit_id}")
        print(f"  - matched_group_ids: {matched_group_ids}")
        print(f"  - matched_brand_ids: {matched_brand_ids}")
        print(f"  - matched_unit_ids: {matched_unit_ids}")
        print(f"  - Available keys in campaign: {list(campaign.keys())}")
    
    return False

# Filtrer les campagnes
filtered_campaigns = []
for campaign in campaigns_data:
    # Debug pour la campagne 284
    if campaign.get('id') == 284:
        print(f"\n🔍 DEBUG: Analyzing campaign 284")
        campaign_belongs_to_scope(campaign, debug_campaign_id=284)
    
    if campaign_belongs_to_scope(campaign):
        # Extraire les IDs directement depuis la campagne
        group_id = campaign.get('group_id')
        brand_id = campaign.get('brand_id')
        unit_id = campaign.get('unit_id')
        
        # Récupérer les noms depuis les DataFrames des groupes/marques/unités
        group_name = None
        if group_id and len(groups_df) > 0 and 'id' in groups_df.columns:
            group_row = groups_df[groups_df['id'] == group_id]
            if len(group_row) > 0 and 'name' in group_row.columns:
                group_name = group_row.iloc[0]['name']
        
        brand_name = None
        if brand_id and len(brands_df) > 0 and 'id' in brands_df.columns:
            brand_row = brands_df[brands_df['id'] == brand_id]
            if len(brand_row) > 0 and 'name' in brand_row.columns:
                brand_name = brand_row.iloc[0]['name']
        
        unit_name = None
        if unit_id and len(units_df) > 0 and 'id' in units_df.columns:
            unit_row = units_df[units_df['id'] == unit_id]
            if len(unit_row) > 0 and 'name' in unit_row.columns:
                unit_name = unit_row.iloc[0]['name']
        
        # Construire le nom hiérarchique
        hierarchy_parts = [p for p in [group_name, brand_name, unit_name] if p]
        hierarchy_name = " > ".join(hierarchy_parts) if hierarchy_parts else "Sans organisation"
        
        # Ajouter les informations enrichies
        campaign_enriched = campaign.copy()
        campaign_enriched['group_name'] = group_name
        campaign_enriched['brand_name'] = brand_name
        campaign_enriched['unit_name'] = unit_name
        campaign_enriched['hierarchy_name'] = hierarchy_name
        
        filtered_campaigns.append(campaign_enriched)

campaigns_df = pd.DataFrame(filtered_campaigns)

print(f"\n✅ {len(campaigns_df)} campagnes filtrées pour le client '{COMPANY_NAME}'")

if len(campaigns_df) > 0:
    print(f"\n📊 Aperçu des campagnes:")
    display_cols = ['id', 'title', 'hierarchy_name']
    available_cols = [col for col in display_cols if col in campaigns_df.columns]
    print(campaigns_df[available_cols].head(10).to_string(index=False))
else:
    print(f"\n⚠️ Aucune campagne trouvée pour le client '{COMPANY_NAME}'")
    print(f"   Vérifiez que le nom du client est correct et qu'il existe des campagnes associées")

# Extraire les IDs de campagnes pour l'étape suivante
campaign_ids = campaigns_df['id'].tolist() if len(campaigns_df) > 0 and 'id' in campaigns_df.columns else []
print(f"\n📋 {len(campaign_ids)} IDs de campagnes à analyser")


📥 Récupération de toutes les campagnes...
✅ 252 campagnes récupérées au total

🔍 DEBUG: Analyzing campaign 284
DEBUG Campaign 284: Matched by unit_id 73.0

✅ 43 campagnes filtrées pour le client 'CRIT INTERIM'

📊 Aperçu des campagnes:
 id                                                title            hierarchy_name
272                          DRAPEUR ENTREES D'AIR (F/H) CRIT INTERIM Aéronautique
273                             AJUSTEURS MONTEURS (F/H) CRIT INTERIM Aéronautique
274                 AJUSTEUR-MONTEUR SOUS ENSEMBLE (F/H) CRIT INTERIM Aéronautique
275                      TECHNICIEN DE MAINTENANCE (F/H) CRIT INTERIM Aéronautique
276            OPERATEUR DE CONTROLE TRIDIMENSIONNEL H/F CRIT INTERIM Aéronautique
277             FORMATION CHAUDRONNIER AERO (F/H) - CDII CRIT INTERIM Aéronautique
278                FORMATION AJUSTEUR-MONTEUR F/H - CDII CRIT INTERIM Aéronautique
279    FORMATION CQPM MONTEUR CABLEUR AERONAUTIQUE (F/H) CRIT INTERIM Aéronautique
280 FORMATION CQPM

## 📝 Étape 7 : Récupération des candidatures

Cette cellule :
- Récupère toutes les candidatures depuis l'API
- Filtre les candidatures appartenant aux campagnes identifiées
- Fusionne les données de candidature avec la hiérarchie des campagnes
- Vérifie qu'il n'y a pas de doublons

**Temps estimé :** 20-40 secondes (peut être plus long si beaucoup de candidatures)


In [73]:
if len(campaign_ids) == 0:
    print("⚠️ Aucune campagne à analyser. Impossible de récupérer les candidatures.")
    applications_df = pd.DataFrame()
else:
    # Récupération de toutes les candidatures
    print("📥 Récupération de toutes les candidatures...")
    applications_raw = fetch_data(ENDPOINTS["applications"], HEADERS)
    
    # Créer un DataFrame comme pour les campagnes
    applications_df_raw = pd.DataFrame(applications_raw) if applications_raw else pd.DataFrame()
    
    # Extraire les données de la colonne 'data' comme pour les campagnes
    applications_df_extracted = extract_data_column(applications_df_raw)
    
    # Convertir en liste de dictionnaires pour le traitement
    applications_data = applications_df_extracted.to_dict('records') if len(applications_df_extracted) > 0 else []
    
    print(f"✅ {len(applications_data)} candidatures récupérées au total")
    
    # Convertir les campaign_ids en int pour la comparaison (une seule fois)
    campaign_ids_int = [int(cid) for cid in campaign_ids if cid is not None]
    
    # Filtrer les candidatures par campaign_id (utiliser directement campaign_id depuis le format raw)
    filtered_applications = []
    for app in applications_data:
        if not isinstance(app, dict):
            continue
        
        # Les endpoints debug utilisent directement campaign_id
        campaign_id = app.get('campaign_id')
        
        # Convertir en int si nécessaire pour la comparaison
        if campaign_id is not None:
            try:
                campaign_id = int(campaign_id)
            except (ValueError, TypeError):
                campaign_id = None
        
        if campaign_id and campaign_id in campaign_ids_int:
            # Trouver les informations de la campagne correspondante
            campaign_info = campaigns_df[campaigns_df['id'] == campaign_id].iloc[0] if len(campaigns_df[campaigns_df['id'] == campaign_id]) > 0 else None
            
            # Enrichir la candidature avec les informations de la campagne
            app_enriched = app.copy()
            if campaign_info is not None:
                app_enriched['campaign_title'] = campaign_info.get('title', 'Sans titre')
                app_enriched['campaign_group_name'] = campaign_info.get('group_name')
                app_enriched['campaign_brand_name'] = campaign_info.get('brand_name')
                app_enriched['campaign_unit_name'] = campaign_info.get('unit_name')
                app_enriched['campaign_hierarchy_name'] = campaign_info.get('hierarchy_name')
            
            filtered_applications.append(app_enriched)
    
    applications_df = pd.DataFrame(filtered_applications)
    
    # Vérifier les doublons
    if len(applications_df) > 0 and 'id' in applications_df.columns:
        duplicates = applications_df.duplicated(subset=['id']).sum()
        if duplicates > 0:
            print(f"⚠️ {duplicates} candidatures en doublon détectées, suppression...")
            applications_df = applications_df.drop_duplicates(subset=['id'])
    
    print(f"\n✅ {len(applications_df)} candidatures filtrées pour le client '{COMPANY_NAME}'")
    
    if len(applications_df) > 0:
        print(f"\n📊 Aperçu des candidatures:")
        display_cols = ['id', 'status', 'source', 'campaign_title']
        available_cols = [col for col in display_cols if col in applications_df.columns]
        if available_cols:
            print(applications_df[available_cols].head(10).to_string(index=False))
        
        # Statistiques rapides
        if 'status' in applications_df.columns:
            print(f"\n📈 Répartition par statut:")
            status_counts = applications_df['status'].value_counts()
            for status, count in status_counts.items():
                print(f"   {status}: {count}")
        
        if 'source' in applications_df.columns:
            print(f"\n📈 Répartition par source:")
            source_counts = applications_df['source'].value_counts()
            for source, count in source_counts.items():
                print(f"   {source}: {count}")
    else:
        print(f"\n⚠️ Aucune candidature trouvée pour les campagnes du client '{COMPANY_NAME}'")


📥 Récupération de toutes les candidatures...
✅ 5590 candidatures récupérées au total

✅ 242 candidatures filtrées pour le client 'CRIT INTERIM'

📊 Aperçu des candidatures:
  id status           source                                       campaign_title
5746    new        hellowork                 AJUSTEUR-MONTEUR SOUS ENSEMBLE (F/H)
5748    new        hellowork            OPERATEUR DE CONTROLE TRIDIMENSIONNEL H/F
5750    new        hellowork                               AJUSTEUR MONTEUR (F/H)
5751    new        hellowork FORMATION CQPM MECANICIEN SYSTEME AERONAUTIQUE (F/H)
5754    new        hellowork                               AJUSTEUR MONTEUR (F/H)
5756    new        hellowork                                          CARISTE F/H
5758    new        hellowork                                          CARISTE F/H
5759 denied cabine cibli job                            PLOMBIER-CHAUFFAGISTE F/H
5760    new        hellowork    FORMATION CQPM MONTEUR CABLEUR AERONAUTIQUE (F/H)
5761    

## 📊 Étape 8 : Calcul des statistiques

Cette section calcule toutes les statistiques de recrutement demandées. Chaque métrique est calculée séparément et stockée dans un DataFrame pour l'export Excel.

### Métriques calculées :
1. **Nombre total de candidatures**
2. **Nombre de candidatures par marque**
3. **Nombre de candidatures par unité**
4. **Nombre de candidatures par campagne**
5. **Origine des candidatures (source)**
6. **Statut des candidatures (status)**


### 1. Nombre total de candidatures

Cette métrique calcule le nombre total de candidatures pour le client.


In [74]:
if len(applications_df) > 0:
    total_applications = len(applications_df)
    stats_total = pd.DataFrame({
        'Métrique': ['Nombre total de candidatures'],
        'Valeur': [total_applications]
    })
    print(f"✅ Nombre total de candidatures: {total_applications}")
else:
    stats_total = pd.DataFrame({
        'Métrique': ['Nombre total de candidatures'],
        'Valeur': [0]
    })
    print("⚠️ Aucune candidature à analyser")


✅ Nombre total de candidatures: 242


### 2. Nombre de candidatures par marque

Cette métrique calcule le nombre de candidatures pour chaque marque du client.


In [ ]:
if len(applications_df) > 0 and 'campaign_brand_name' in applications_df.columns:
    # Remplacer les NaN par "Sans marque (direct au groupe)" pour inclure toutes les candidatures
    df_with_brand = applications_df.copy()
    df_with_brand['campaign_brand_name'] = df_with_brand['campaign_brand_name'].fillna('Sans marque (direct au groupe)')
    
    stats_by_brand = df_with_brand.groupby('campaign_brand_name').size().reset_index(name='Nombre de candidatures')
    stats_by_brand = stats_by_brand.sort_values('Nombre de candidatures', ascending=False)
    stats_by_brand.columns = ['Marque', 'Nombre de candidatures']
    print(f"✅ Statistiques par marque calculées ({len(stats_by_brand)} marques)")
    print(stats_by_brand.to_string(index=False))
    
    # Vérification: afficher le total pour s'assurer qu'on n'a rien perdu
    total_in_brands = stats_by_brand['Nombre de candidatures'].sum()
    total_applications = len(applications_df)
    if total_in_brands != total_applications:
        print(f"⚠️ Attention: {total_applications - total_in_brands} candidatures manquantes dans le calcul par marque")
    else:
        print(f"✅ Vérification: Toutes les {total_applications} candidatures sont comptées")
else:
    stats_by_brand = pd.DataFrame({
        'Marque': ['Aucune'],
        'Nombre de candidatures': [0]
    })
    print("⚠️ Aucune donnée de marque disponible")


✅ Statistiques par marque calculées (2 marques)
                   Marque  Nombre de candidatures
CRIT INTERIM Région Ouest                      94
CRIT INTERIM Aéronautique                      47


### 3. Nombre de candidatures par unité

Cette métrique calcule le nombre de candidatures pour chaque unité du client.


In [ ]:
if len(applications_df) > 0 and 'campaign_unit_name' in applications_df.columns:
    # Remplacer les NaN par "Sans unité (direct au groupe/marque)" pour inclure toutes les candidatures
    df_with_unit = applications_df.copy()
    df_with_unit['campaign_unit_name'] = df_with_unit['campaign_unit_name'].fillna('Sans unité (direct au groupe/marque)')
    
    stats_by_unit = df_with_unit.groupby('campaign_unit_name').size().reset_index(name='Nombre de candidatures')
    stats_by_unit = stats_by_unit.sort_values('Nombre de candidatures', ascending=False)
    stats_by_unit.columns = ['Unité', 'Nombre de candidatures']
    print(f"✅ Statistiques par unité calculées ({len(stats_by_unit)} unités)")
    print(stats_by_unit.to_string(index=False))
    
    # Vérification: afficher le total pour s'assurer qu'on n'a rien perdu
    total_in_units = stats_by_unit['Nombre de candidatures'].sum()
    total_applications = len(applications_df)
    if total_in_units != total_applications:
        print(f"⚠️ Attention: {total_applications - total_in_units} candidatures manquantes dans le calcul par unité")
    else:
        print(f"✅ Vérification: Toutes les {total_applications} candidatures sont comptées")
else:
    stats_by_unit = pd.DataFrame({
        'Unité': ['Aucune'],
        'Nombre de candidatures': [0]
    })
    print("⚠️ Aucune donnée d'unité disponible")


✅ Statistiques par unité calculées (4 unités)
                                                   Unité  Nombre de candidatures
                           CRIT INTERIM - Saint-Herblain                      81
                                     CRIT INTERIM - Rezé                      66
            CRIT INTERIM - Direction Régionale CRIT BPDL                      42
CRIT INTERIM - Cabinet Experts & Cadres (Saint-Herblain)                       6


### 4. Nombre de candidatures par campagne

Cette métrique calcule le nombre de candidatures pour chaque campagne du client.


In [77]:
if len(applications_df) > 0 and 'campaign_title' in applications_df.columns:
    stats_by_campaign = applications_df.groupby('campaign_title').size().reset_index(name='Nombre de candidatures')
    stats_by_campaign = stats_by_campaign.sort_values('Nombre de candidatures', ascending=False)
    stats_by_campaign.columns = ['Campagne', 'Nombre de candidatures']
    print(f"✅ Statistiques par campagne calculées ({len(stats_by_campaign)} campagnes)")
    print(f"\nTop 10 des campagnes:")
    print(stats_by_campaign.head(10).to_string(index=False))
else:
    stats_by_campaign = pd.DataFrame({
        'Campagne': ['Aucune'],
        'Nombre de candidatures': [0]
    })
    print("⚠️ Aucune donnée de campagne disponible")


✅ Statistiques par campagne calculées (32 campagnes)

Top 10 des campagnes:
                                            Campagne  Nombre de candidatures
                                         CARISTE F/H                      43
                            RECRUTEUR DE TALENTS H/F                      42
                      MANOEUVRE TRAVAUX PUBLICS F//H                      19
                            OPERATEUR LOGISTIQUE F/H                      16
  AGENT DE TRI F/H - GRANDCHAMPS DES FONTAINES 44119                      12
               FORMATION AJUSTEUR-MONTEUR F/H - CDII                      10
                           PLOMBIER-CHAUFFAGISTE F/H                      10
FORMATION CQPM MECANICIEN SYSTEME AERONAUTIQUE (F/H)                       9
                            MAGASINIER CARISTE (F/H)                       6
                       MACON TRADITIONNEL N3P2 (H/F)                       6


### 5. Origine des candidatures (source)

Cette métrique analyse d'où viennent les candidatures (source d'origine).


In [78]:
if len(applications_df) > 0 and 'source' in applications_df.columns:
    stats_by_source = applications_df.groupby('source').size().reset_index(name='Nombre de candidatures')
    stats_by_source = stats_by_source.sort_values('Nombre de candidatures', ascending=False)
    stats_by_source.columns = ['Source', 'Nombre de candidatures']
    # Calculer le pourcentage
    total = stats_by_source['Nombre de candidatures'].sum()
    stats_by_source['Pourcentage'] = (stats_by_source['Nombre de candidatures'] / total * 100).round(2)
    print(f"✅ Statistiques par source calculées ({len(stats_by_source)} sources)")
    print(stats_by_source.to_string(index=False))
else:
    stats_by_source = pd.DataFrame({
        'Source': ['Aucune'],
        'Nombre de candidatures': [0],
        'Pourcentage': [0]
    })
    print("⚠️ Aucune donnée de source disponible")


✅ Statistiques par source calculées (3 sources)
            Source  Nombre de candidatures  Pourcentage
         hellowork                     210        86.78
  cabine cibli job                      31        12.81
recruiter_cvtheque                       1         0.41


### 6. Statut des candidatures (status)

Cette métrique analyse le statut actuel de toutes les candidatures.


In [79]:
if len(applications_df) > 0 and 'status' in applications_df.columns:
    stats_by_status = applications_df.groupby('status').size().reset_index(name='Nombre de candidatures')
    stats_by_status = stats_by_status.sort_values('Nombre de candidatures', ascending=False)
    stats_by_status.columns = ['Statut', 'Nombre de candidatures']
    # Calculer le pourcentage
    total = stats_by_status['Nombre de candidatures'].sum()
    stats_by_status['Pourcentage'] = (stats_by_status['Nombre de candidatures'] / total * 100).round(2)
    print(f"✅ Statistiques par statut calculées ({len(stats_by_status)} statuts)")
    print(stats_by_status.to_string(index=False))
else:
    stats_by_status = pd.DataFrame({
        'Statut': ['Aucune'],
        'Nombre de candidatures': [0],
        'Pourcentage': [0]
    })
    print("⚠️ Aucune donnée de statut disponible")


✅ Statistiques par statut calculées (6 statuts)
           Statut  Nombre de candidatures  Pourcentage
              new                     168        69.42
          on_hold                      40        16.53
           denied                      28        11.57
appointment_taken                       3         1.24
   appointment_do                       2         0.83
        shortlist                       1         0.41


## 💾 Étape 9 : Export vers Excel

Cette cellule génère un fichier Excel avec toutes les statistiques calculées. Chaque métrique est exportée dans une feuille séparée pour faciliter l'analyse.

**Fichier généré :** `client_recruitment_stats_[NOM_CLIENT]_[TIMESTAMP].xlsx`

**Contenu :**
- Une feuille par métrique
- Colonnes ajustées automatiquement
- En-têtes formatés
- Timestamp de génération


In [ ]:
def auto_adjust_column_widths(ws):
    """Ajuste automatiquement la largeur des colonnes"""
    for column in ws.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)  # Max 50 caractères
        ws.column_dimensions[column_letter].width = adjusted_width

def style_header_row(ws, row_num):
    """Applique un style aux en-têtes"""
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF")
    header_alignment = Alignment(horizontal="center", vertical="center")
    
    for cell in ws[row_num]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment

# Créer le workbook
wb = Workbook()
wb.remove(wb.active)  # Supprimer la feuille par défaut

# Générer le nom du fichier avec timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
safe_company_name = COMPANY_NAME.replace(" ", "_").replace("/", "_")[:30]
filename = f"client_recruitment_stats_{safe_company_name}_{timestamp}.xlsx"

# Feuille 1: Total des candidatures
ws_total = wb.create_sheet("Total Candidatures")
ws_total.append(['Métrique', 'Valeur'])
ws_total.append(['Nombre total de candidatures', len(applications_df) if len(applications_df) > 0 else 0])
ws_total.append(['Date de génération', datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws_total.append(['Client analysé', COMPANY_NAME])
style_header_row(ws_total, 1)
auto_adjust_column_widths(ws_total)

# Feuille 2: Par marque
ws_brand = wb.create_sheet("Par Marque")
if len(stats_by_brand) > 0:
    ws_brand.append(list(stats_by_brand.columns))
    for _, row in stats_by_brand.iterrows():
        ws_brand.append(list(row))
style_header_row(ws_brand, 1)
auto_adjust_column_widths(ws_brand)

# Feuille 3: Par unité
ws_unit = wb.create_sheet("Par Unité")
if len(stats_by_unit) > 0:
    ws_unit.append(list(stats_by_unit.columns))
    for _, row in stats_by_unit.iterrows():
        ws_unit.append(list(row))
style_header_row(ws_unit, 1)
auto_adjust_column_widths(ws_unit)

# Feuille 4: Par campagne
ws_campaign = wb.create_sheet("Par Campagne")
if len(stats_by_campaign) > 0:
    ws_campaign.append(list(stats_by_campaign.columns))
    for _, row in stats_by_campaign.iterrows():
        ws_campaign.append(list(row))
style_header_row(ws_campaign, 1)
auto_adjust_column_widths(ws_campaign)

# Feuille 5: Par source
ws_source = wb.create_sheet("Par Source")
if len(stats_by_source) > 0:
    ws_source.append(list(stats_by_source.columns))
    for _, row in stats_by_source.iterrows():
        ws_source.append(list(row))
style_header_row(ws_source, 1)
auto_adjust_column_widths(ws_source)

# Feuille 6: Par statut
ws_status = wb.create_sheet("Par Statut")
if len(stats_by_status) > 0:
    ws_status.append(list(stats_by_status.columns))
    for _, row in stats_by_status.iterrows():
        ws_status.append(list(row))
style_header_row(ws_status, 1)
auto_adjust_column_widths(ws_status)

# Feuille 7: Par groupe (si disponible) - S'assurer d'inclure toutes les candidatures
if len(applications_df) > 0 and 'campaign_group_name' in applications_df.columns:
    # Remplacer les NaN par le nom du client pour inclure toutes les candidatures
    df_with_group = applications_df.copy()
    df_with_group['campaign_group_name'] = df_with_group['campaign_group_name'].fillna(COMPANY_NAME)
    
    stats_by_group = df_with_group.groupby('campaign_group_name').size().reset_index(name='Nombre de candidatures')
    stats_by_group = stats_by_group.sort_values('Nombre de candidatures', ascending=False)
    stats_by_group.columns = ['Groupe', 'Nombre de candidatures']
    
    # Vérification
    total_in_groups = stats_by_group['Nombre de candidatures'].sum()
    total_applications = len(applications_df)
    if total_in_groups != total_applications:
        print(f"⚠️ Attention: {total_applications - total_in_groups} candidatures manquantes dans le calcul par groupe")
    
    ws_group = wb.create_sheet("Par Groupe")
    ws_group.append(list(stats_by_group.columns))
    for _, row in stats_by_group.iterrows():
        ws_group.append(list(row))
    style_header_row(ws_group, 1)
    auto_adjust_column_widths(ws_group)

# Feuille 8: Source par Statut (tableau croisé)
if len(applications_df) > 0 and 'source' in applications_df.columns and 'status' in applications_df.columns:
    cross_source_status = pd.crosstab(applications_df['source'], applications_df['status'], margins=True)
    cross_source_status.index.name = 'Source'
    cross_source_status.columns.name = 'Statut'
    
    ws_cross_source_status = wb.create_sheet("Source x Statut")
    # Écrire les en-têtes
    headers = ['Source'] + list(cross_source_status.columns)
    ws_cross_source_status.append(headers)
    # Écrire les données
    for idx, row in cross_source_status.iterrows():
        ws_cross_source_status.append([idx] + list(row))
    style_header_row(ws_cross_source_status, 1)
    auto_adjust_column_widths(ws_cross_source_status)

# Feuille 9: Marque par Source (tableau croisé)
if len(applications_df) > 0 and 'campaign_brand_name' in applications_df.columns and 'source' in applications_df.columns:
    cross_brand_source = pd.crosstab(applications_df['campaign_brand_name'], applications_df['source'], margins=True)
    cross_brand_source.index.name = 'Marque'
    cross_brand_source.columns.name = 'Source'
    
    ws_cross_brand_source = wb.create_sheet("Marque x Source")
    headers = ['Marque'] + list(cross_brand_source.columns)
    ws_cross_brand_source.append(headers)
    for idx, row in cross_brand_source.iterrows():
        ws_cross_brand_source.append([idx] + list(row))
    style_header_row(ws_cross_brand_source, 1)
    auto_adjust_column_widths(ws_cross_brand_source)

# Feuille 10: Marque par Statut (tableau croisé)
if len(applications_df) > 0 and 'campaign_brand_name' in applications_df.columns and 'status' in applications_df.columns:
    cross_brand_status = pd.crosstab(applications_df['campaign_brand_name'], applications_df['status'], margins=True)
    cross_brand_status.index.name = 'Marque'
    cross_brand_status.columns.name = 'Statut'
    
    ws_cross_brand_status = wb.create_sheet("Marque x Statut")
    headers = ['Marque'] + list(cross_brand_status.columns)
    ws_cross_brand_status.append(headers)
    for idx, row in cross_brand_status.iterrows():
        ws_cross_brand_status.append([idx] + list(row))
    style_header_row(ws_cross_brand_status, 1)
    auto_adjust_column_widths(ws_cross_brand_status)

# Feuille 11: Unité par Source (tableau croisé)
if len(applications_df) > 0 and 'campaign_unit_name' in applications_df.columns and 'source' in applications_df.columns:
    cross_unit_source = pd.crosstab(applications_df['campaign_unit_name'].fillna('Sans unité'), applications_df['source'], margins=True)
    cross_unit_source.index.name = 'Unité'
    cross_unit_source.columns.name = 'Source'
    
    ws_cross_unit_source = wb.create_sheet("Unité x Source")
    headers = ['Unité'] + list(cross_unit_source.columns)
    ws_cross_unit_source.append(headers)
    for idx, row in cross_unit_source.iterrows():
        ws_cross_unit_source.append([idx] + list(row))
    style_header_row(ws_cross_unit_source, 1)
    auto_adjust_column_widths(ws_cross_unit_source)

# Feuille 12: Unité par Statut (tableau croisé)
if len(applications_df) > 0 and 'campaign_unit_name' in applications_df.columns and 'status' in applications_df.columns:
    cross_unit_status = pd.crosstab(applications_df['campaign_unit_name'].fillna('Sans unité'), applications_df['status'], margins=True)
    cross_unit_status.index.name = 'Unité'
    cross_unit_status.columns.name = 'Statut'
    
    ws_cross_unit_status = wb.create_sheet("Unité x Statut")
    headers = ['Unité'] + list(cross_unit_status.columns)
    ws_cross_unit_status.append(headers)
    for idx, row in cross_unit_status.iterrows():
        ws_cross_unit_status.append([idx] + list(row))
    style_header_row(ws_cross_unit_status, 1)
    auto_adjust_column_widths(ws_cross_unit_status)

# Feuille 13: Campagne par Source (tableau croisé) - Top 20 seulement pour éviter les fichiers trop grands
if len(applications_df) > 0 and 'campaign_title' in applications_df.columns and 'source' in applications_df.columns:
    # Prendre les top 20 campagnes par nombre de candidatures
    top_campaigns = applications_df['campaign_title'].value_counts().head(20).index.tolist()
    df_top_campaigns = applications_df[applications_df['campaign_title'].isin(top_campaigns)]
    cross_campaign_source = pd.crosstab(df_top_campaigns['campaign_title'], df_top_campaigns['source'], margins=True)
    cross_campaign_source.index.name = 'Campagne'
    cross_campaign_source.columns.name = 'Source'
    
    ws_cross_campaign_source = wb.create_sheet("Campagne x Source")
    headers = ['Campagne'] + list(cross_campaign_source.columns)
    ws_cross_campaign_source.append(headers)
    for idx, row in cross_campaign_source.iterrows():
        ws_cross_campaign_source.append([idx] + list(row))
    style_header_row(ws_cross_campaign_source, 1)
    auto_adjust_column_widths(ws_cross_campaign_source)

# Feuille 14: Campagne par Statut (tableau croisé) - Top 20 seulement
if len(applications_df) > 0 and 'campaign_title' in applications_df.columns and 'status' in applications_df.columns:
    top_campaigns = applications_df['campaign_title'].value_counts().head(20).index.tolist()
    df_top_campaigns = applications_df[applications_df['campaign_title'].isin(top_campaigns)]
    cross_campaign_status = pd.crosstab(df_top_campaigns['campaign_title'], df_top_campaigns['status'], margins=True)
    cross_campaign_status.index.name = 'Campagne'
    cross_campaign_status.columns.name = 'Statut'
    
    ws_cross_campaign_status = wb.create_sheet("Campagne x Statut")
    headers = ['Campagne'] + list(cross_campaign_status.columns)
    ws_cross_campaign_status.append(headers)
    for idx, row in cross_campaign_status.iterrows():
        ws_cross_campaign_status.append([idx] + list(row))
    style_header_row(ws_cross_campaign_status, 1)
    auto_adjust_column_widths(ws_cross_campaign_status)

# Feuille Hiérarchique: Structure complète avec totaux et statuts
if len(applications_df) > 0 and 'status' in applications_df.columns:
    ws_hierarchical = wb.create_sheet("Hiérarchie Complète", 0)  # Mettre en première position
    
    # Obtenir tous les statuts uniques pour les colonnes
    all_statuses = sorted(applications_df['status'].unique().tolist())
    
    # En-têtes
    headers = ['Entité', 'Type', 'Total'] + all_statuses
    ws_hierarchical.append(headers)
    style_header_row(ws_hierarchical, 1)
    
    row_idx = 2
    
    # Fonction pour calculer les stats par statut
    def get_status_counts(df):
        """Retourne un dictionnaire avec les counts par statut"""
        if len(df) == 0:
            return {status: 0 for status in all_statuses}
        status_counts = df['status'].value_counts().to_dict()
        return {status: status_counts.get(status, 0) for status in all_statuses}
    
    # Niveau 1: Groupe (toutes les candidatures)
    group_name = applications_df['campaign_group_name'].iloc[0] if len(applications_df) > 0 and 'campaign_group_name' in applications_df.columns else COMPANY_NAME
    group_total = len(applications_df)
    group_status_counts = get_status_counts(applications_df)
    
    # Ligne TOTAL Groupe
    group_row = row_idx
    ws_hierarchical.append([group_name, 'GROUPE', group_total] + [group_status_counts[s] for s in all_statuses])
    # Style en gras pour le groupe
    for col in range(1, len(headers) + 1):
        cell = ws_hierarchical.cell(row=group_row, column=col)
        cell.font = Font(bold=True)
    row_idx += 1
    
    # Niveau 2: Par Marque
    if 'campaign_brand_name' in applications_df.columns:
        brands = applications_df['campaign_brand_name'].dropna().unique()
        brand_start_rows = {}
        
        for brand in sorted(brands):
            brand_df = applications_df[applications_df['campaign_brand_name'] == brand]
            brand_total = len(brand_df)
            brand_status_counts = get_status_counts(brand_df)
            
            brand_row = row_idx
            brand_start_rows[brand] = row_idx
            ws_hierarchical.append([brand, 'MARQUE', brand_total] + [brand_status_counts[s] for s in all_statuses])
            # Style en gras pour la marque
            for col in range(1, len(headers) + 1):
                cell = ws_hierarchical.cell(row=brand_row, column=col)
                cell.font = Font(bold=True)
            row_idx += 1
            
            # Niveau 3: Par Unité (sous cette marque)
            if 'campaign_unit_name' in applications_df.columns:
                units = brand_df['campaign_unit_name'].dropna().unique()
                unit_start_rows = {}
                
                for unit in sorted(units):
                    unit_df = brand_df[brand_df['campaign_unit_name'] == unit]
                    unit_total = len(unit_df)
                    unit_status_counts = get_status_counts(unit_df)
                    
                    unit_row = row_idx
                    unit_start_rows[unit] = row_idx
                    ws_hierarchical.append([unit, 'UNITÉ', unit_total] + [unit_status_counts[s] for s in all_statuses])
                    # Style italique pour l'unité
                    for col in range(1, len(headers) + 1):
                        cell = ws_hierarchical.cell(row=unit_row, column=col)
                        cell.font = Font(italic=True)
                    row_idx += 1
                    
                    # Niveau 4: Par Campagne (sous cette unité)
                    if 'campaign_title' in applications_df.columns:
                        campaigns = unit_df['campaign_title'].dropna().unique()
                        campaign_start_row = row_idx
                        
                        for campaign in sorted(campaigns):
                            campaign_df = unit_df[unit_df['campaign_title'] == campaign]
                            campaign_total = len(campaign_df)
                            campaign_status_counts = get_status_counts(campaign_df)
                            
                            ws_hierarchical.append([campaign, 'CAMPAGNE', campaign_total] + [campaign_status_counts[s] for s in all_statuses])
                            row_idx += 1
                        
                        # Grouper les campagnes sous l'unité
                        if row_idx > campaign_start_row:
                            ws_hierarchical.row_dimensions.group(campaign_start_row, row_idx - 1, outline_level=3, hidden=False)
                
                # Grouper les unités sous la marque
                if len(units) > 0:
                    first_unit_row = min(unit_start_rows.values()) if unit_start_rows else None
                    last_unit_row = row_idx - 1
                    if first_unit_row:
                        ws_hierarchical.row_dimensions.group(first_unit_row, last_unit_row, outline_level=2, hidden=False)
            
            # Grouper les marques sous le groupe
            if row_idx > brand_row + 1:
                ws_hierarchical.row_dimensions.group(brand_row + 1, row_idx - 1, outline_level=1, hidden=False)
    
    # Gérer les candidatures directement au groupe (sans marque/unité)
    if 'campaign_brand_name' in applications_df.columns:
        direct_to_group = applications_df[applications_df['campaign_brand_name'].isna()]
        if len(direct_to_group) > 0:
            direct_total = len(direct_to_group)
            direct_status_counts = get_status_counts(direct_to_group)
            
            direct_row = row_idx
            ws_hierarchical.append([f"{group_name} (direct)", 'DIRECT', direct_total] + [direct_status_counts[s] for s in all_statuses])
            # Style en gras pour les candidatures directes
            for col in range(1, len(headers) + 1):
                cell = ws_hierarchical.cell(row=direct_row, column=col)
                cell.font = Font(bold=True, color="FF0000")  # Rouge pour les directes
            row_idx += 1
    
    # Grouper toutes les marques sous le groupe
    if row_idx > group_row + 1:
        ws_hierarchical.row_dimensions.group(group_row + 1, row_idx - 1, outline_level=0, hidden=False)
    
    # Ajuster les largeurs des colonnes
    auto_adjust_column_widths(ws_hierarchical)
    
    # Freezer la première ligne
    ws_hierarchical.freeze_panes = 'A2'
    
    # Activer le groupage
    ws_hierarchical.sheet_properties.outlinePr.summaryBelow = True
    ws_hierarchical.sheet_properties.outlinePr.summaryRight = False

# Sauvegarder le fichier
wb.save(filename)

# Compter le nombre de feuilles créées
sheet_count = len(wb.sheetnames)
print(f"✅ Fichier Excel généré: {filename}")
print(f"\n📊 Feuilles créées ({sheet_count} au total):")
if len(applications_df) > 0 and 'status' in applications_df.columns:
    print(f"   ⭐ Hiérarchie Complète (structure Groupe > Marque > Unité > Campagne avec statuts)")
print(f"   1. Total Candidatures")
print(f"   2. Par Marque ({len(stats_by_brand)} lignes)")
print(f"   3. Par Unité ({len(stats_by_unit)} lignes)")
print(f"   4. Par Campagne ({len(stats_by_campaign)} lignes)")
print(f"   5. Par Source ({len(stats_by_source)} lignes)")
print(f"   6. Par Statut ({len(stats_by_status)} lignes)")
if len(applications_df) > 0 and 'campaign_group_name' in applications_df.columns:
    print(f"   7. Par Groupe")
if len(applications_df) > 0 and 'source' in applications_df.columns and 'status' in applications_df.columns:
    print(f"   - Source x Statut (tableau croisé)")
if len(applications_df) > 0 and 'campaign_brand_name' in applications_df.columns and 'source' in applications_df.columns:
    print(f"   - Marque x Source (tableau croisé)")
if len(applications_df) > 0 and 'campaign_brand_name' in applications_df.columns and 'status' in applications_df.columns:
    print(f"   - Marque x Statut (tableau croisé)")
if len(applications_df) > 0 and 'campaign_unit_name' in applications_df.columns and 'source' in applications_df.columns:
    print(f"   - Unité x Source (tableau croisé)")
if len(applications_df) > 0 and 'campaign_unit_name' in applications_df.columns and 'status' in applications_df.columns:
    print(f"   - Unité x Statut (tableau croisé)")
if len(applications_df) > 0 and 'campaign_title' in applications_df.columns and 'source' in applications_df.columns:
    print(f"   - Campagne x Source (Top 20, tableau croisé)")
if len(applications_df) > 0 and 'campaign_title' in applications_df.columns and 'status' in applications_df.columns:
    print(f"   - Campagne x Statut (Top 20, tableau croisé)")
print(f"\n💡 Le fichier est prêt à être téléchargé depuis le panneau de fichiers à gauche.")


✅ Fichier Excel généré: client_recruitment_stats_CRIT_INTERIM_20260105_223718.xlsx

📊 Feuilles créées (14 au total):
   1. Total Candidatures
   2. Par Marque (2 lignes)
   3. Par Unité (4 lignes)
   4. Par Campagne (32 lignes)
   5. Par Source (3 lignes)
   6. Par Statut (6 lignes)
   7. Par Groupe
   - Source x Statut (tableau croisé)
   - Marque x Source (tableau croisé)
   - Marque x Statut (tableau croisé)
   - Unité x Source (tableau croisé)
   - Unité x Statut (tableau croisé)
   - Campagne x Source (Top 20, tableau croisé)
   - Campagne x Statut (Top 20, tableau croisé)

💡 Le fichier est prêt à être téléchargé depuis le panneau de fichiers à gauche.


---

## ✅ C'est terminé !

### 📁 Fichier généré

Le fichier Excel `client_recruitment_stats_[NOM_CLIENT]_[TIMESTAMP].xlsx` a été créé avec toutes les statistiques.

### 📥 Pour télécharger le fichier

1. Cliquez sur l'icône **📁 "Fichiers"** dans la barre latérale gauche
2. Trouvez le fichier Excel (nom commençant par `client_recruitment_stats_`)
3. Faites un **clic droit** sur le fichier
4. Sélectionnez **"Télécharger"** (Download)

### 🔄 Pour analyser un autre client

1. **Modifiez la variable `COMPANY_NAME`** dans la cellule de configuration (Étape 3)
2. **Réexécutez toutes les cellules** :
   - Menu **"Runtime"** → **"Restart runtime"**
   - Menu **"Runtime"** → **"Run all"**

### 📊 Interprétation des résultats

**Feuille "Total Candidatures" :**
- Affiche le nombre total de candidatures pour le client

**Feuille "Par Marque" :**
- Montre la répartition des candidatures par marque
- Utile pour identifier quelles marques génèrent le plus de candidatures

**Feuille "Par Unité" :**
- Montre la répartition des candidatures par unité
- Utile pour identifier les unités les plus actives

**Feuille "Par Campagne" :**
- Montre le nombre de candidatures par campagne
- Utile pour identifier les campagnes les plus performantes

**Feuille "Par Source" :**
- Montre d'où viennent les candidatures (ex: "cabine cibli job", "hellowork", etc.)
- Inclut les pourcentages pour voir la répartition

**Feuille "Par Statut" :**
- Montre le statut actuel des candidatures (ex: "new", "denied", "accepted", etc.)
- Inclut les pourcentages pour voir la répartition

---

**Merci d'avoir utilisé ce notebook ! 🎉**
